In [1]:
import json

path_processed_data = "../data/processed_data/"

with open(path_processed_data + "farms.geojson", "r") as file:
    farms_data = json.load(file)

with open(path_processed_data + "enriched_buildings_with_farms.geojson", "r") as file:
    buildings_data = json.load(file)

In [2]:
farms_dict = {feature["id"]: feature["properties"] for feature in farms_data["features"]}

# Enrich the buildings GeoJSON with farm categories and products
for building in buildings_data["features"]:
    for nearby_farm in building["properties"].get("nearby_farms", []):
        farm_id = nearby_farm["farm_id"]
        if farm_id in farms_dict:
            farm_properties = farms_dict[farm_id]
            nearby_farm["categories"] = [item["category"] for item in farm_properties.get("lebensmittel", [])]
            nearby_farm["products"] = [
                product
                for item in farm_properties.get("lebensmittel", [])
                for product in item["products"]
            ]

In [3]:
top_n = 1000

for building in buildings_data["features"]:
    total_products = sum(
        len(farm["products"]) for farm in building["properties"].get("nearby_farms", [])
    )
    building["properties"]["total_products"] = total_products

sorted_buildings = sorted(
    buildings_data["features"],
    key=lambda x: x["properties"]["total_products"],
    reverse=True,
)

top_buildings = sorted_buildings[:top_n]

buildings_data["features"] = top_buildings

output_filename = f"top_{top_n}_buildings_with_farms_details.geojson"
with open(path_processed_data + output_filename, "w") as file:
    json.dump(buildings_data, file, ensure_ascii=False, indent=4)

print(f"Saved the enriched buildings data with nearby farms details to {output_filename}")

Saved the enriched buildings data with nearby farms details to top_1000_buildings_with_farms_details.geojson
